# Prediksi Hilal

In [ ]:
# _jengkolrebus
# Juni, 2020
# Curup, Bengkulu

Koordinat kota:
- Yogyakarta: 7.7956 S, 110.3695 E, <a href="https://www.google.com/search?q=koordinat+yogyakarta&rlz=1C1CHBF_enID904ID904&oq=koordinat+yogyakarta&aqs=chrome..69i57.5458j0j7&sourceid=chrome&ie=UTF-8">sumber</a>
- Sabang: 5.892386 N, 95.323706 E, sumber: google maps
- Merauke: 8.499324 S, 140.404902 E, sumber: google maps

In [2]:
from skyfield import almanac
from skyfield.api import load, Topos
from skyfield.units import Angle

from datetime import datetime, timedelta
from pytz import timezone

from ipywidgets import widgets, interactive
from IPython.display import display, HTML

import pandas as pd
import calendar

In [3]:
jkt = timezone('Asia/Jakarta')
ts = load.timescale()
e = load('de421.bsp')


In [4]:
YOG = Topos('7.7956 S','110.3695 E')
SAB = Topos('5.892386 N', '95.323706 E')
MER = Topos('8.499324 S', '140.404902 E')

In [5]:
class var:
    KOTA = [YOG, SAB, MER]
    conj = []
    sunset = []
    moon_alt = []
    moon_az = []
    moon_d = []
    sun_alt = []
    sun_az = []
    sun_d = []
    elong = []
    moonset = []
    usia_bulan = []
    ir =[]

In [6]:
def c():
    var.conj.clear()
    var.sunset.clear()
    var.moon_alt.clear()
    var.moon_az.clear()
    var.moon_d.clear()
    var.sun_alt.clear()
    var.sun_az.clear()
    var.sun_d.clear()
    var.elong.clear()
    var.moonset.clear()
    var.usia_bulan.clear()
    var.ir.clear()

# Metode untuk membandingkan dengan Imkan Rukyat
def imkanRukyat():
    alt = var.moon_alt
    elong = var.elong
    usia_bulan = var.usia_bulan
    for i, j, k in zip(alt, elong, usia_bulan):
        if(i.degrees >= 2 and j.degrees >= 3 and (timedelta.total_seconds(k)/3600) > 8):
            var.ir.append(u'\u2714')
        else:
            var.ir.append(u'\u2718')
    
def objPos(t, obj, i):
    loc = e['earth'] + i
    astrometric = loc.at(t).observe(e[obj])
    alt, az, d = astrometric.apparent().altaz()
    
    return astrometric, alt, az, d.km

def usiaBulan():
    for i, j in zip(var.conj, var.moonset):
        t0 = j
        t1 = i
        dt = t0-t1
#         (h, m, s) = str(dt).split(':')
#         result = (int(h)) + (int(m) / 60) + (float(s)/3600)
#         dt = timedelta.total_seconds(dt)/3600
        var.usia_bulan.append(dt)

def moonSet(t, i):
    konj = t
    t = t.utc
    t0 = ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])
    t1 = ts.utc(t[0], t[1], t[2]+1, t[3], t[4], t[5])
    f = almanac.risings_and_settings(e, e['moon'], i)
    t, y = almanac.find_discrete(t0, t1, f)
    
    for ti, yi in zip(t, y):
        if(yi==False):
            var.moonset.append(ti.astimezone(jkt))
        else:
            pass

def find(t0, t1, i):
    c()
    
    # Konjungsi
    t0 = ts.utc(t0)
    t1 = ts.utc(t1)
    f = almanac.oppositions_conjunctions(e, e['moon'])
    t, y = almanac.find_discrete(t0, t1, f)
    
    for ti, yi in zip(t, y):
        if(yi == 1):
            var.conj.append(ti.astimezone(jkt))
            moonSet(ti, i)
            t = ti.utc
            t0 = ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])
            t1 = ts.utc(t[0], t[1], t[2]+1, t[3], t[4], t[5])
            t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(e, i))
            
            for ti, yi in zip(t, y):
                if(yi == False):
                    var.sunset.append(ti.astimezone(jkt))
                    
                    moon = objPos(ti, 'moon', i)
                    sun = objPos(ti, 'sun', i)
                    var.moon_alt.append(moon[1])
                    var.moon_az.append(moon[2])
                    var.moon_d.append(moon[3])
                    
                    var.sun_alt.append(sun[1])
                    var.sun_az.append(sun[2])
                    var.sun_d.append(sun[3])
                    
                    elong = moon[0].separation_from(sun[0])
                    var.elong.append(elong)
                    
        else:
            pass
    usiaBulan()
    imkanRukyat()
    
    print()
    print(i)
    table = list(zip(var.conj, var.sunset, 
                     var.moon_alt, var.moon_az, 
                     var.sun_alt, var.sun_az,
                     var.elong, var.usia_bulan,
                     var.ir))
    df = pd.DataFrame(table, columns=['Waktu Konjungsi', 'Waktu Sunset',
              'Alt Bulan', 'Az Bulan',
              'Alt Matahari', 'Az Bulan',
              'Elong', 'Usia Bulan', 'Imkan Rukyat'])
    df.index+=1
    display(df)
    print()
    

def findAll(t0, t1):
    for i in var.KOTA:
        find(t0, t1, i)

In [7]:
hari_ini = datetime.now().date()
days_in_month = calendar.monthrange(hari_ini.year, hari_ini.month)[1]
bulan_depan = hari_ini + timedelta(days_in_month)

dari = widgets.DatePicker(value=hari_ini,
    description='Dari:',
    disabled=False
)
sampai = widgets.DatePicker(value=bulan_depan,
    description='Sampai:',
    disabled=False
)

interactive(find, t0=dari, t1=sampai)

ValueError: cannot find widget or abbreviation for argument: 'i'